# Этап 1: Векторный поиск

## Описание
Имеется структурированная база знаний, содержащая инструкции и правила по взаимодействию оператора Пункта выдачи заказов (ПВЗ) с платформой, клиентом, товарами и описания других бизнес-процессов.

Необходимо реализовать систему, которая по вопросу оператора ПВЗ будет находить релевантные фрагменты базы знаний.

## Исходные данные
1. **chunks (1).csv** - база знаний, разбитая на чанки
   - Содержит заголовок и содержание каждого фрагмента
2. **train_data.csv** - размеченные тренировочные данные
   - Содержит вопросы операторов и соответствующие им чанки

### Метрики качества:
- recall@1
- recall@3
- recall@5

## План реализации проекта

### 1. Подготовка данных
   - Загрузка и первичный анализ данных
   - Предобработка текста

### 2. Разработка базовой модели
   - Реализация TF-IDF векторизации
   - Создание функции косинусного сходства
   - Базовая оценка качества

### 3. Улучшение модели
   - Эксперименты с различными методами векторизации
   - Оценка качества и сравнение
   - Применение функции сравнения нового вопроса с уже имеющимися

### 4. Создание сервиса
   - Разработка класса ChunkFinder
   - Реализация FastAPI для класса ChunkFinder по поиску чанков на вопросы

In [1]:
from typing import Dict, List
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Загрузим стоп слова для обработки русской речи
nltk.download('stopwords')
nltk.download('punkt_tab')
russian_stopwords = stopwords.words('russian')

# Инициализация лемматизатора
mystem = Mystem()
# Инициализация векторизатора
vectorizer = TfidfVectorizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Artem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### 1. Подготовка данных

In [2]:
# Загрузка данных
chunks_df = pd.read_csv('chunks (1).csv')
train_df = pd.read_csv('train_data.csv')

In [ ]:
# Посмотрю как выглядит содержимое
print(chunks_df.shape)
chunks_df.head()

In [ ]:
print(train_df.shape)
train_df.head()

### Оценка содержимого

Видно, что у нас текст имеет разные регистры, разные формы речи, местоимения. 

Все эти особенности речи влияют в отрациательную стоону для поиска по вектрной схожести, таким образом нужно все слова привести к нижнему регистру, нужно провести лемматизацию для получения стандартной морфологии слова, а так же удаления местоимений и лишних знаков. 

Я протестирую 2 версии по качеству векторизации с объединением заголовков и без. С одной строны заголовки несут в себе контекст и ключевые слова, с другой стороны они могут повторяться. 

In [5]:
# Ф-ция для стандартной очистки слов в тексте
def preprocess_text(text: str) -> str:
    # Приведем к нижнему регистру
    text = text.lower()
    # Удалим спец. символы
    text = re.sub(r'[^\w\s]', ' ', text)
    # Токенизация
    tokens = word_tokenize(text)
    # Удаляем стоп-слова
    tokens = [token for token in tokens if token not in russian_stopwords]
    # Лемматизация
    lemmatized_tokens = mystem.lemmatize(' '.join(tokens))
    # Удаляем пробелы и пустые строки после лемматизации
    lemmatized_tokens = [token.strip() for token in lemmatized_tokens if token.strip()]
    return ' '.join(lemmatized_tokens)

In [6]:
# Создам столбец в chunks_df, который будет содержать заголовок и чанк для полноты поиска
chunks_df['full_text'] = chunks_df['Headers'] + ' ' + chunks_df['Chunk']

In [7]:
# Получим преобразованный столбец
chunks_df['processed_text'] = chunks_df['full_text'].apply(preprocess_text)
# chunks_df['processed_text'] = chunks_df['Chunk'].apply(preprocess_text)

In [ ]:
# Оценим результат
chunks_df.head()

### 2. Разработка базовой модели

In [9]:
# Создаем TF-IDF матрицу для чанков
chunks_vectors = vectorizer.fit_transform(chunks_df['processed_text'])

In [ ]:
train_df.head()

In [11]:
# Ф-ция для оценки на основе косинусного сходства
def find_relevant_chunks(question: str, top_k: int=5) -> list:
    # Предобработка вопроса
    processed_question = preprocess_text(question)
    # Векторизуем вопрос
    question_vector = vectorizer.transform([processed_question])
    # Вычисляем косинусное сходство
    similarities = cosine_similarity(question_vector, chunks_vectors).flatten()
    # Получаем индексы top_k наиболее релевантных чанков
    top_indices = similarities.argsort()[-top_k:][::-1]
    # Формируем результат
    results = []
    for idx in top_indices:
        results.append({
            'header': chunks_df.iloc[idx]['Headers'],
            'chunk': chunks_df.iloc[idx]['Chunk'],
            'similarity_score': similarities[idx]
        })
    
    return results

In [9]:
# Функция для оценки модели
def evaluate_model(test_data: pd.DataFrame, k_values: list=[1, 3, 5]) -> Dict[str, float]:
    recalls = {k: [] for k in k_values}
    
    for _, row in test_data.iterrows():
        question = row['Question']
        correct_chunk = row['Chunk']
        
        results = find_relevant_chunks(question, max(k_values))
        
        for k in k_values:
            top_k_chunks = [r['chunk'] for r in results[:k]]
            recall = 1 if correct_chunk in top_k_chunks else 0
            recalls[k].append(recall)
    
    return {f'recall@{k}': np.mean(recalls[k]) for k in k_values}

In [13]:
evaluate_model(train_df)

{'recall@1': np.float64(0.2714285714285714),
 'recall@3': np.float64(0.45285714285714285),
 'recall@5': np.float64(0.5342857142857143)}

In [14]:
# headers + chunks + edit lemma + processed_question
{'recall@1': np.float64(0.2714285714285714),
 'recall@3': np.float64(0.45285714285714285),
 'recall@5': np.float64(0.5342857142857143)}

{'recall@1': np.float64(0.2714285714285714),
 'recall@3': np.float64(0.45285714285714285),
 'recall@5': np.float64(0.5342857142857143)}

In [15]:
# chunks
{'recall@1': np.float64(0.22),
 'recall@3': np.float64(0.37714285714285717),
 'recall@5': np.float64(0.47285714285714286)}

{'recall@1': np.float64(0.22),
 'recall@3': np.float64(0.37714285714285717),
 'recall@5': np.float64(0.47285714285714286)}

### 3.1 Улучшение модели. Попробую BERT в надежде улучшить качество модели

BERT и TF-IDF работают по-разному:

- BERT понимает семантический смысл и контекст
- TF-IDF работает на уровне частоты слов

In [53]:
from sentence_transformers import SentenceTransformer

# Загрузка данных
chunks_df = pd.read_csv('chunks (1).csv')
train_df = pd.read_csv('train_data.csv')

In [54]:
# Используем многоязычную модель BERT
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [63]:
# Текст с заголовком
chunks_df['full_text'] = chunks_df['Headers'] + ' ' + chunks_df['Chunk']

In [64]:
# Предварительно вычисляем эмбеддинги
embeddings = model.encode(chunks_df['full_text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 10/10 [00:09<00:00,  1.08it/s]


In [65]:
def find_relevant_chunks(question, top_k=5):
    # Получаем эмбеддинг вопроса
    question_embedding = model.encode([question])
    
    # Вычисляем косинусное сходство
    similarities = cosine_similarity(question_embedding, embeddings).flatten()
    
    # Получаем топ-k результатов
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    return [
        {
            'header': chunks_df.iloc[idx]['Headers'],
            'chunk': chunks_df.iloc[idx]['Chunk'],
            'similarity_score': float(similarities[idx])
        }
        for idx in top_indices
    ]

In [66]:
evaluate_model(train_df)

{'recall@1': np.float64(0.21714285714285714),
 'recall@3': np.float64(0.3757142857142857),
 'recall@5': np.float64(0.4471428571428571)}

In [ ]:
# Просто чанки
{'recall@1': np.float64(0.2042857142857143),
 'recall@3': np.float64(0.3357142857142857),
 'recall@5': np.float64(0.3942857142857143)}

In [ ]:
# Просто заголовки и чанки
{'recall@1': np.float64(0.21714285714285714),
 'recall@3': np.float64(0.3757142857142857),
 'recall@5': np.float64(0.4471428571428571)}

In [22]:
# Улучшенное объединение текста с весами, удвоенная важность заголовков
{'recall@1': np.float64(0.18714285714285714),
 'recall@3': np.float64(0.29285714285714287),
 'recall@5': np.float64(0.37)}

{'recall@1': np.float64(0.18714285714285714),
 'recall@3': np.float64(0.29285714285714287),
 'recall@5': np.float64(0.37)}

### 3.2 Еще попробую модель от sbert_large_nlu_ru нашел на habr

In [23]:
import torch
from transformers import AutoTokenizer, AutoModel

# Загрузка данных
chunks_df = pd.read_csv('chunks (1).csv')
train_df = pd.read_csv('train_data.csv')

In [24]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

In [39]:
# Текст с заголовком
chunks_df['full_text'] = chunks_df['Headers'] + ' ' + chunks_df['Chunk']

In [ ]:
#  model.cuda()  # GPU

In [40]:
# Ф-ция для получения эмбеддингов через BERT
def embed_bert_cls(texts, model, tokenizer, max_length=512):
    embeddings = []
    for text in texts:
        t = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in t.items()})
        embedding = model_output.last_hidden_state[:, 0, :]
        embedding = torch.nn.functional.normalize(embedding)
        embeddings.append(embedding[0].cpu().numpy())
    return embeddings

embeddings = embed_bert_cls(chunks_df['full_text'].to_list(), model, tokenizer)

In [41]:
# Ф-ция для оценки на основе косинусного сходства между вопросом и чанками
def find_relevant_chunks(question, top_k=5):
    # Получаем эмбеддинг вопроса
    question_embedding = embed_bert_cls([question], model, tokenizer)[0]
    
    # Вычисляем косинусное сходство
    similarities = cosine_similarity([question_embedding], embeddings).flatten()
    
    # Получаем топ-k результатов
    top_indices = similarities.argsort()[-top_k:][::-1]
    
    return [
        {
            'header': chunks_df.iloc[idx]['Headers'],
            'chunk': chunks_df.iloc[idx]['Chunk'],
            'similarity_score': float(similarities[idx])
        }
        for idx in top_indices
    ]

In [42]:
evaluate_model(train_df)

{'recall@1': np.float64(0.02142857142857143),
 'recall@3': np.float64(0.054285714285714284),
 'recall@5': np.float64(0.08)}

Результат гораздо хуже чем у первой модели, буду работать с вариантом 3.1. Протестирую вариант с очисткой текста и лемматизацией.

### 3.3 Улучшение модели из 3.1. Попробую скомбинировать

Веса 0.7 и 0.3 дают преимущество BERT, потому что:
 - BERT лучше улавливает смысловые связи
 - TF-IDF помогает "заякорить" результат на ключевых словах

In [4]:
from sentence_transformers import SentenceTransformer

# Загрузка данных
chunks_df = pd.read_csv('chunks (1).csv')
train_df = pd.read_csv('train_data.csv')

# Используем многоязычную модель BERT
bert_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
# TF-IDF векторайзер
tfidf = TfidfVectorizer(min_df=2, max_df=0.95, ngram_range=(1, 2))

c:\Users\Artem\Desktop\DS\Norvik_workplace\vector_search\.venvwin\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
chunks_df[['Headers', 'Chunk']].head()

In [ ]:
train_df.head()

In [5]:
# Ф-ция для стандартной очистки слов в тексте
def preprocess_text_tfidf(text: str) -> str:
    # Приведем к нижнему регистру
    text = text.lower()
    # Удалим спец. символы
    text = re.sub(r'[^\w\s]', ' ', text)
    # Токенизация
    tokens = word_tokenize(text)
    # Удаляем стоп-слова
    tokens = [token for token in tokens if token not in russian_stopwords]
    # Лемматизация
    lemmatized_tokens = mystem.lemmatize(' '.join(tokens))
    # Удаляем пробелы и пустые строки после лемматизации
    lemmatized_tokens = [token.strip() for token in lemmatized_tokens if token.strip()]
    return ' '.join(lemmatized_tokens)

In [6]:
chunks_df['full_text'] = chunks_df['Headers'] + ' ' + chunks_df['Chunk']
# Получим преобразованный столбец
chunks_df['processed_text'] = chunks_df['full_text'].apply(preprocess_text_tfidf)

# Создаем TF-IDF матрицу для чанков
embeddings_tfidf = tfidf.fit_transform(chunks_df['processed_text'])
# Вычисляем эмбеддинги для BERT
embeddings_bert = bert_model.encode(chunks_df['full_text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [ ]:
def find_relevant_chunks(question: str, top_k: int = 5) -> List[Dict]:
    """
    Улучшенная версия поиска с сохранением эффективного подхода
    
    Аргументы:
        question: вопрос для поиска
        top_k: количество возвращаемых результатов
        bert_weight: вес для BERT скора
        tfidf_weight: вес для TF-IDF скора
    """
    # BERT similarities с нормализацией
    question_bert_emb = bert_model.encode([question])
    bert_similarities = cosine_similarity(question_bert_emb, embeddings_bert).flatten()
    
    # TF-IDF similarities с предобработкой
    processed_question = preprocess_text_tfidf(question)
    question_tfidf = tfidf.transform([processed_question])
    tfidf_similarities = cosine_similarity(question_tfidf, embeddings_tfidf).flatten()
    
    # Комбинируем scores с весами
    combined_similarities = (bert_similarities + tfidf_similarities)
    
    # Получаем топ результаты
    top_indices = combined_similarities.argsort()[-top_k:][::-1]
    
    # Формируем результаты с дополнительной информацией
    results = []
    for idx in top_indices:
        bert_contribution = bert_similarities[idx]
        tfidf_contribution = tfidf_similarities[idx]
        
        results.append({
            'header': chunks_df.iloc[idx]['Headers'],
            'chunk': chunks_df.iloc[idx]['Chunk'],
            'similarity_score': float(combined_similarities[idx]),
            'bert_score': float(bert_similarities[idx]),
            'tfidf_score': float(tfidf_similarities[idx]),
            'bert_contribution': float(bert_contribution),
            'tfidf_contribution': float(tfidf_contribution)
        })
    
    return results

In [16]:
evaluate_model(train_df)

{'recall@1': np.float64(0.29),
 'recall@3': np.float64(0.5071428571428571),
 'recall@5': np.float64(0.5771428571428572)}

In [ ]:
# комбинированный подход combined_similarities = 0.7 * bert_similarities + 0.3 * tfidf_similarities
{'recall@1': np.float64(0.2842857142857143),
 'recall@3': np.float64(0.4614285714285714),
 'recall@5': np.float64(0.5471428571428572)}

In [ ]:
# не комбинированный подход, просто выбор лучших результатов
{'recall@1': np.float64(0.21857142857142858),
 'recall@3': np.float64(0.37714285714285717),
 'recall@5': np.float64(0.45)}

In [ ]:
# комбинированный подход combined_similarities = 1 * bert_similarities + 1 * tfidf_similarities
{'recall@1': np.float64(0.29),
 'recall@3': np.float64(0.5071428571428571),
 'recall@5': np.float64(0.5771428571428572)}

In [ ]:
# комбинированный подход combined_similarities = 0.3 * bert_similarities + 0.7 * tfidf_similarities
{'recall@1': np.float64(0.2814285714285714),
 'recall@3': np.float64(0.5085714285714286),
 'recall@5': np.float64(0.5857142857142857)}

In [ ]:
# комбинированный подход combined_similarities = 0.8 * bert_similarities + 1 * tfidf_similarities
{'recall@1': np.float64(0.2842857142857143),
 'recall@3': np.float64(0.5085714285714286),
 'recall@5': np.float64(0.5842857142857143)}

In [ ]:
# комбинированный подход combined_similarities = 0.6 * bert_similarities + 1 * tfidf_similarities
{'recall@1': np.float64(0.2785714285714286),
 'recall@3': np.float64(0.5114285714285715),
 'recall@5': np.float64(0.5985714285714285)}

In [ ]:
# комбинированный подход combined_similarities = 0.4 * bert_similarities + 1 * tfidf_similarities
{'recall@1': np.float64(0.2785714285714286),
 'recall@3': np.float64(0.5042857142857143),
 'recall@5': np.float64(0.5928571428571429)}

In [ ]:
# комбинированный подход combined_similarities = 0.9 * bert_similarities + 1 * tfidf_similarities
{'recall@1': np.float64(0.28714285714285714),
 'recall@3': np.float64(0.5042857142857143),
 'recall@5': np.float64(0.5814285714285714)}

Веса не дают сильного эффекта но усложняют логику. Лучше использовать простой подход.

Так же я попробую использовать имеющиеся вопросы.
Я думаю, что если собрать хорошую выборку с вопросами, то она уже будет содержать в себе ответы на 90% вопросов. 
Так что на вопрос по тестовой выборке я буду искать вопросы в тренировочном датасете, а уже потом подключать модель.

In [ ]:
# Создание эмбеддингов для тренировочных вопросов
train_questions_bert = bert_model.encode(train_df['Question'].tolist(), show_progress_bar=True)

In [ ]:
def find_similar_training_question(self, question: str) -> tuple:
    """
    Поиск похожего вопроса в тренировочной выборке

    Аргументы:
        question: текст вопроса
        
    Возвращает:
        tuple: (индекс найденного вопроса, значение схожести) или (None, 0)
    """
    # Получаем эмбеддинг для нового вопроса
    question_embedding = self.bert_model.encode([question])
    
    # Вычисляем схожесть с тренировочными вопросами
    similarities = cosine_similarity(question_embedding, self.train_questions_bert)[0]
    
    # Находим максимальную схожесть и соответствующий индекс
    max_similarity = np.max(similarities)
    max_similarity_idx = np.argmax(similarities)
    
    # Если схожесть выше порога, возвращаем результат
    if max_similarity >= self.bert_similarity_threshold:
        return max_similarity_idx, max_similarity
        
    return None, max_similarity

Это и будет мой финальный вариант. Таким образом это комбинированный варант. 

1) Поиск по максимально похожему вопросу из train, 

2) поиска по семантике BERT а так же буквальному совпадению по словам это tfidf.